# molPX Di-Ala example
<pre> 
Guillermo Pérez-Hernández  guille.perez@fu-berlin.de 
</pre>
   
In this notebook we will be using a trajectory of Di-Ala-peptide to easily identify conformations in the Ramachandran plot.

In [ ]:
from os.path import exists
import molpx
from matplotlib import pylab as plt
%matplotlib ipympl

import pyemma
import numpy as np

## Start from files on disk

In [ ]:
top = molpx._molpxdir(join='notebooks/data/ala2.pdb')
# What data do we  have?
if exists('/group/ag_cmb/scratch/gph82/Di-Ala-nbdata/ala2.dcd'):
    MD_trajfiles = ['/group/ag_cmb/scratch/gph82/Di-Ala-nbdata/ala2.dcd'] #long trajectory
elif exists('/home/guille/ala2.dcd'):
    MD_trajfiles = ['/home/guille/ala2.dcd'] # extra for Stralsund
else:
    MD_trajfiles = [molpx._molpxdir(join='notebooks/data/ala2.mini.xtc')] #short trajectory

## Featurize to Ramachandran $(\phi,\psi)$-pairs with `PyEMMA`

In [ ]:
feat = pyemma.coordinates.featurizer(top)
feat.add_backbone_torsions()
src  = pyemma.coordinates.source(MD_trajfiles, features=feat)
Y = src.get_output()

## Visualize a FES and the molecular structures behind it
Execute the following cell and click either on the FES or on the slidebar

In [ ]:
ax, fig, iwd, data_sample, geom, widget_box = molpx.visualize.FES(MD_trajfiles,
                                                                  top,    
                                                                  Y,             
                                                                  #proj_idxs=[1],
                                                                  nbins=50,
                                                                  n_overlays=5,
                                                                  proj_labels=['$\phi$', 
                                                                               '$\psi$'],
                                                                  atom_selection="symbol != H", 
                                                                  sticky=True, 
                                                                  #color_list='random'
                                                                 )
widget_box

## Visualize trajectories, FES and molecular structures

In [ ]:
__, myfig, iwd, __, ibox = molpx.visualize.traj(MD_trajfiles,     
                                          top,                                                                                                                              
                                          Y,                                        
                                          plot_FES = True,                                        
                                          #dt = dt*1e-6, tunits='ms',                                           
                                          max_frames=10000,
                                          proj_idxs=[0, 1],
                                          panel_height=2,    
                                          proj_labels=['$\phi$', '$\psi$']
                          )
myfig.tight_layout()
ibox

# Intermediate steps: using molPX to generate a regspace sample of the data
See the documentation of `molpx.generate.sample` to find out about all possible options:
```
molpx.generate.sample(MD_trajectories, MD_top, projected_trajectories, proj_idxs=[0, 1], n_points=100, n_geom_samples=1, keep_all_samples=False, proj_stride=1, verbose=False, return_data=False)
```

In [ ]:
data_sample, geoms = molpx.generate.sample(MD_trajfiles,                                            
                                           top, 
                                           Y,                                            
                                           n_points=200   ,
                                            n_geom_samples=5,
                                           keep_all_samples=True,
                                    )
data_sample.shape, geoms

## Link the PDF plot with the sampled structures and visually explore the FES 
Click either on the plot or on the widget slidebar: they're connected! 

In [ ]:
# Replot the FES
plt.figure(figsize=(4,4))
h, (x,y) = np.histogramdd(np.vstack(Y)[:,:2], bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

# Create the linked widget
linked_wdg, __ = molpx.visualize.sample(data_sample, 
                              geoms,                      
                              plt.gca(), 
                              clear_lines=True,
                              #sticky=True,                              
                            )
linked_wdg

# Paths samples along the different projections (=axis)

In [ ]:
paths_dict, idata = molpx.generate.projection_paths(MD_trajfiles, 
                                                    top, 
                                                    Y, 
                                                    n_points=50,
                                                    proj_idxs=[0,1],
                                                    n_projs=3,
                                                    proj_dim = 3, 
                                                    verbose=False, 
                                        )

In [ ]:
# Choose the coordinate and the tyep of path
coord = 1
path_type = 'min_rmsd'
#path_type = 'min_disp'
igeom = paths_dict[coord][path_type]["geom"]
ipath = paths_dict[coord][path_type]["proj"]

# Choose the proj_idxs for the path and the FES 
# to be shown
proj_idxs = [0,1]

In [ ]:
plt.figure(figsize=(5,5))
h, (x,y) = np.histogramdd(np.vstack(Y)[:,proj_idxs], bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

linked_NGL_wdg, linked_ax_widget = molpx.visualize.sample(ipath[:,proj_idxs], 
                                    igeom,                             
                                    plt.gca(), 
                                    clear_lines=True,
                                    n_smooth = 2, 
                                    plot_path=True,                                                            
                                    #radius=True,
                            )
linked_NGL_wdg

# Let's do TICA and try to look a the correlations in a TICA analysis

In [ ]:
feat = pyemma.coordinates.featurizer(top)
#feat.add_backbone_torsions(cossin=True)
feat.add_distances(feat.topology.select('symbol != H'))
src  = pyemma.coordinates.source(MD_trajfiles, features=feat)
tica = pyemma.coordinates.tica(src, lag=np.int(src.trajectory_lengths()/3000))
Y_tica = tica.get_output()

In [ ]:
ax, fig, iwd, data_sample, geom, widget_box = molpx.visualize.FES(MD_trajfiles,
                                                                  top,                                                       
                                                                  Y_tica,                                                       
                                                                  n_overlays=5,
                                                                  atom_selection='backbone',
                                                                  #sticky=True,
                                                                  #color_list='rand'
                                          )
widget_box

In [ ]:
__, myfig, iwd, __, ibox = molpx.visualize.traj(MD_trajfiles,     
                                          top,                                                                                                                              
                                          Y_tica,                                        
                                          plot_FES = True,                                        
                                          #dt = dt*1e-6, tunits='ms',                                           
                                          max_frames=10000,
                                          #proj_idxs=[0,1],
                                          panel_height=2,                                              
                                          projection=tica
                          )
ibox

In [ ]:
paths_dict, idata = molpx.generate.projection_paths(MD_trajfiles, 
                                                    top, 
                                                    Y_tica, 
                                                    n_points=50,
                                                    proj_idxs=[0,1],
                                                    n_projs=2,
                                                    proj_dim = 2, 
                                                    verbose=False, 
                                        )

In [ ]:
# Choose the coordinate and the type of path
coord = 0
path_type = 'min_rmsd'
#path_type = 'min_disp'
igeom = paths_dict[coord][path_type]["geom"]
ipath = paths_dict[coord][path_type]["proj"]

# Choose the proj_idxs for the path and the FES 
# to be shown
proj_idxs = [0,1]

In [ ]:
plt.figure(figsize=(4,4))
h, (x,y) = np.histogramdd(np.vstack(Y_tica)[:,proj_idxs], bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

linked_wdg, axes_widget = molpx.visualize.sample(ipath[:,proj_idxs], 
                                    igeom,
                                    plt.gca(), 
                                    clear_lines=True,
                                    n_smooth = 1, 
                                    plot_path=True,   
                                   )
# You can even choose to add the correlations a posteriori                                         
molpx.visualize.correlations(tica, widget=linked_wdg, proj_idxs=0)
linked_wdg.center_view()
linked_wdg